<a href="https://colab.research.google.com/github/putririgita14/bangkit/blob/main/Problem_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ======================================================================================================
# PROBLEM A3
#
# Build a classifier for the Human or Horse Dataset with Transfer Learning.
# The test will expect it to classify binary classes.
# Note that all the layers in the pre-trained model are non-trainable.
# Do not use lambda layers in your model.
#
# The horse-or-human dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
# Inception_v3, pre-trained model used in this problem is developed by Google.
#
# Desired accuracy and validation_accuracy > 97%.
# =======================================================================================================

# Install keras_preprocessing
!pip install keras_preprocessing

import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3


def solution_A3():
    inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    urllib.request.urlretrieve(
        inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
    local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

    # Load the InceptionV3 model without the top layer
    pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                    include_top=False,
                                    weights=None)
    pre_trained_model.load_weights(local_weights_file)

    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('mixed7')
    last_output = last_layer.output

    # Add a GlobalAveragePooling2D layer
    x =  layers.GlobalAveragePooling2D()(last_output)
    # Add a fully connected layer with 1024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(1, activation='sigmoid')(x)

    model = Model(pre_trained_model.input, x)

    model.compile(optimizer=RMSprop(lr=0.0001),
                  loss='binary_crossentropy',
                  metrics=['acc'])

    data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
    local_file = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/horse-or-human')
    zip_ref.close()

    data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
    local_file = 'validation-horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/validation-horse-or-human')
    zip_ref.close()

    train_dir = 'data/horse-or-human'
    validation_dir = 'data/validation-horse-or-human'

    train_datagen = ImageDataGenerator(rescale=1.0/255.0)
    val_datagen = ImageDataGenerator(rescale=1.0/255.0)

    # YOUR IMAGE SIZE SHOULD BE 150x150
    train_generator=  train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary'
    )

    validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary'
    )

    # Train the model
    model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=20,
        verbose=2
    )

    return model

# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_A3()
    model.save("model_A3.h5")


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/20
52/52 - 55s - loss: 0.0629 - acc: 0.9766 - val_loss: 0.0334 - val_acc: 0.9883 - 55s/epoch - 1s/step
Epoch 2/20
52/52 - 50s - loss: 0.0095 - acc: 0.9971 - val_loss: 0.0101 - val_acc: 1.0000 - 50s/epoch - 952ms/step
Epoch 3/20
52/52 - 48s - loss: 0.0028 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 1.0000 - 48s/epoch - 923ms/step
Epoch 4/20
52/52 - 52s - loss: 0.0020 - acc: 1.0000 - val_loss: 0.0075 - val_acc: 1.0000 - 52s/epoch - 991ms/step
Epoch 5/20
52/52 - 50s - loss: 5.7190e-04 - acc: 1.0000 - val_loss: 0.0061 - val_acc: 1.0000 - 50s/epoch - 956ms/step
Epoch 6/20
52/52 - 49s - loss: 4.5019e-04 - acc: 1.0000 - val_loss: 0.0079 - val_acc: 1.0000 - 49s/epoch - 935ms/step
Epoch 7/20
52/52 - 50s - loss: 2.7032e-04 - acc: 1.0000 - val_loss: 0.0068 - val_acc: 1.0000 - 50s/epoch - 953ms/step
Epoch 8/20
52/52 - 48s - loss: 2.1050e-04 - acc: 1.0000 - val_loss: 0.0048 - val_acc: 1.0000 - 48s/epoch - 915

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
